In [26]:
import json
import pandas as pd
import csv
import numpy as np
import MeCab
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from collections import OrderedDict

In [27]:
def readpdCSV(file_pass, file_name, file_type):
    print(file_name)
    df = pd.read_csv(file_pass+file_name+file_type, dtype={'body': 'str', 'category': 'str'}, encoding="utf-8")

    return df.dropna()

In [28]:
def septfIdf(df, name):
    body = np.array([])
    pick_term = []
    shihyo_words = []
    for row in df["body"]:
        if(row != "NaN" or row != "nan"):
            noun = sepNoun(row)
            body = np.append(body, noun)

    vectorizer = TfidfVectorizer(use_idf=False)
    vecs = vectorizer.fit_transform(body)
    terms = vectorizer.get_feature_names()
    for vec in vecs.toarray():
        for i, t in zip(vec, terms):
            # if(0.1 <= i <= 0.3):
            if(i >= 0.35):
                flag = 0
                if(pick_term == []):
                    pick_term.append(t)
                    continue
                for v in pick_term:
                    if(t == v):
                        flag = 1
                if(flag == 0):
                    pick_term.append(t)

    print(name)
    print(pick_term)
    print(len(pick_term))
    print("-"*40)
    return pick_term


In [29]:
def sepNoun(data):
    mt = MeCab.Tagger('mecabrc')
    mt.parse("")
    # print(data)
    node = mt.parseToNode(data)
    # print(node.surface)
    return getNouns(node)

def getNouns(node):
    words = []
    word_stack = ""
    while node:
        noun = node.feature.split(",")
        word = node.surface
        if(checkNoun(noun)):
            word_stack += word
        if(word_stack != "" and noun[0] != '名詞'):
            if(checkNum(word_stack)):
                words.append(word_stack)
            word_stack = ""
        node = node.next
    words = " ".join(words)
    return words

def sepData(df, sepfile_names):
    arr = [[], [], [], [], [], [], [], [], [], [], [], []]
    for data in df:
        # print(data)
        for num, article in data.iterrows():
            # print(article["category"])
            cat_list = article["category"]
            # print(cat_list)
            flag = 0
            for i in range(11):
                # print(cat_list[i])
                if(cat_list[i] == "1"):
                    flag = 1
                    if(arr[i] == []):
                        arr[i].append(["head", "date", "body", "category"])
                    arr[i].append([article["head"], article["date"], article["body"], cat_list])

            if(flag == 0):
                if(arr[11] == []):
                    arr[11].append(["head", "date", "body", "category"])
                arr[11].append([article["head"], article["date"], article["body"], cat_list])

    # print(arr[11])
    for i in range(len(sepfile_names)):
        wc.writeCSV(arr[i], "../csv_sep/"+sepfile_names[i])

In [30]:
def checkNoun(noun):
    zyogai = ["副詞可能", "非自立", "代名詞", "助詞類接続"]
    if(noun[0] == '名詞'):
        if(noun[1] in zyogai):
            return False
        else:
            return True
        
def checkNum(word_stack):
    mt = MeCab.Tagger()
    mt.parse("")

    node = mt.parseToNode(word_stack)
    while node:
        noun = node.feature.split(",")
        if(noun[1] == '数'):
            return False
        node = node.next

    return True

In [31]:
def countVect(df, file_names):
    body = np.array([])
    for d in df:
        body = np.append(body, " ".join(d))

    count_vectorizer = CountVectorizer()
    vecs = count_vectorizer.fit_transform(body)
    features = count_vectorizer.get_feature_names()
    # print(features)
    # print(max_count)
    term_countlist = []
    terms = []
    vec = vecs.toarray()
    # print(vec)
    # print(len(vec))
    # それぞれの単語の出現回数をリスト化
    for t, i in zip(features, range(len(features))):
        count = 0
        for num in range(len(file_names)):
            # print(vec[num][i])
            count += vec[num][i]
        term_countlist.append(count)
    # print(term_countlist)

    need_term = []

    for count, f in zip(term_countlist, features):
        if(int(count) < 2):
            need_term.append(f)
    # print(need_term)

    terms_pick = []
    for d in df:
        pick = []
        for term in d:
            for nt in need_term:
                if(str(term) == str(nt)):
                    pick.append(term)

        terms_pick.append(pick)

    # print(terms_pick)

    return terms_pick


In [39]:
import time
def writeJSON(d):
    with open("./json/word_dict2.json", "w") as f:
        json.dump(d, f, ensure_ascii=False)
    time.sleep(2.0)

In [33]:
file_type = ".csv"
sepfile_names = ["keiki", "bukka", "kinri", "money", "boueki", "koyou", "kojin", "kainyu", "oil", "kabu", "saiken", "youjin", "seiji"]

# それぞれの予想材料ごとに名詞分割して重み付け
# csvの読み込み
df = []
data = []


In [34]:
# youjinとseijiは別で作成
for name in sepfile_names:
    print(name)
    df.append(readpdCSV("./csv_sep/", name, file_type))
# print(df)
print("---csvの読み込み完了---")

keiki
keiki
bukka
bukka
kinri
kinri
money
money
boueki
boueki
koyou
koyou
kojin
kojin
kainyu
kainyu
oil
oil
kabu
kabu
saiken
saiken
youjin
youjin
seiji
seiji
---csvの読み込み完了---


In [35]:
for d, name in zip(df, sepfile_names):
#     print(d["body"])
    data.append(septfIdf(d, name))

keiki
['約束', 'インフレ', '意識', '懸念', '米原油先物', '経済', '指摘', '総裁', '高水準', 'gdp', '予測不可能', '国内', '状況', '生産', '関連', '雇用', '減速', '下方修正', '住宅着工件数', '国内景気', '基本的', '景気回復基調', '減少', '額賀財務相', '完全', '抑制', 'nar', '低下', '住宅販売保留指数', '全米リアルター協会', '前月比', '発表', '中国', '原因', '最悪', '直面', '食品価格', '高騰', 'ユーロ圏域内', '上方修正', '伸び率改定値', '前回発表', 'データ', '住宅ローン', '入手', '可能', '同社', '差し押さえ', '延滞件数', '管理', 'ゴールドマンサックス', 'ディフェンシブ銘柄', '米国内', '見方', 'frb', 'リスク', '方針', '景気減速', '注視', '講演', '一致指数', '基調判断', '指数', '構成', '見通し', '鉱工業生産', 'ecb', '放置', '賃金上昇', '食品エネルギー高', '経済成長', '売り上げ', '左右', '景気動向', '若者向け衣料', '鈍化傾向', '高級品', '判断', '実感', '景気ウォッチャー調査', '景気回復', '小売売上高', '米商務省', '伸び', '自動車', '米連邦準備理事会', 'エコノミスト調査', 'ロイター', '予想', '設備稼働率', '上昇', '増加', '申請', '水準', 'エコノミスト', '住宅着工許可件数', '低迷', '住宅市場', '推移', '数カ月', '住宅', '在庫', '着工', '足かせ', '敏感', '銘柄', '利下げ', '先行き判断', '期待', '維持', '企業部門', '先行き', '季節調整済み', '英国立統計局', 'リスク要因', '最大', '国家統計局', '通年', 'インフレ加速', '名目ベース', '中古住宅販売', '対策', '必要', '相当', '規模', '認識', '金融政策運営', '基調', '強調', '時点', '景気対策', '株安', '歯

2970
----------------------------------------
bukka
['ナスダック指数', 'ハイテク株', '値上がり', '成績', '通年', 'エクソン', 'シェブロン', '年間', 'cpi', '消費者物価指数', '高水準', 'ニューヨーク', '値下がり', '東京', 'nar', '低下', '住宅販売保留指数', '全米リアルター協会', '前月比', '発表', 'ハイオク', '前回', '灯油店頭価格', '中長期的', '理解', 'タイト化', '上昇', '石油製品', '需給ギャップ', '物価上昇率', '物価見通し', '購入', '上昇率', '推定', '見通し', 'ge', 'ibm', '伸び', '円買い', '拡大', '日本', '限定的', '大幅', '食品', '傾向', '問題', '実体経済', '影響', '株価', '状況', 'p金融株指数', '割合', '指数', '構成銘柄', '商品', '買い控え', '全面安', 'リスク要因', 'ユーロ圏', '予定', '小売売上高', '新規失業保険申請件数', '消費者信頼感指数', '米ミシガン', '速報値', '鉱工業生産', 'cgpi', '企業物価指数', '伸び率', '国内企業物価指数', '日銀', '速報', '株安', 'ガソリン', 'マインド', '悪化', '指摘', '消費者', '足元', '食料品', 'ppi', 'インフレ期待', '波及', '中国', '中国国家統計局', '住宅着工件数', '実質所得', '米国', '経済指標', '複数', '余剰資金', '原油', '受け皿', '商品市場', '意識', '貴金属', '金融緩和', 'frb', '懸念', 'cb', 'コンファレンスボード', '下方修正', '米大手民間調査機関', 'カトリーナ', 'ハリケーン', '期待指数', 'ケースシラー', 'スタンダードアンドプアーズ', 'データ', '価格動向', '米一戸建て中古住宅価格', '困難', '恐れ', '総裁', '過度', 'イラク開戦時', 'エコノミスト予想', '水準', 'インフレ圧力', '一因', '商品市況

1347
----------------------------------------
money
['クロス', 'ファンド勢', '付け商い', '動き', '大手証券', '見方', '資金移動', 'cd', 'マネーサプライ', '伸び', '加速', '通貨供給量', '低迷続き', '兆し', '国内株式ファンド', '資金流入', 'nri', '国内株式', '流入', '記録', '問題', '銀行', '影響', '状況', '取り組み', '可能', '排出削減', '支援', '省エネ努力', '程度', '資金供給', '途上国', '協調', '日米欧', '前月末', '外貨準備高', '発表', '財務省', '中国本土', '国別', '日本', '比較可能', '首位', '更新', '最高', 'アセットマネジメント部門', '資金流出', 'インフレ', '中国', '原因', '金利', '中小企業', '国内外', '株価下落', '海外顧客', '減少', '目減り', '要因', '資産価値', 'インフレ懸念', 'ココア', 'コーヒー', '上昇', '商品市場', '小麦', '背景', '物価', '強化', '抑制', '過剰流動性', 'ロシア', '台湾', '外国', '大台', '伸び率', '拡大', '日銀', 'frb', '信用リスク', '大量', '実施', '市場', '思惑', '米連邦準備理事会', '信用市場', '動揺', '増大', '払しょく', '資金繰り懸念', '資金需要', 'ドル', 'ポジション', 'コモディティ', 'ドル安', 'ニューヨーク連銀', 'ベアースターンズ', '理由', '資金繰り悪化', '好調', '新興国', '選別', 'おう盛', '供給', '反映', '応募', '公定歩合', '円滑', '実行', '必要', '買い入れ', '金融市場調節上', '長期国債', '欧州市場', '欧州金融機関', '不安定化継続', '期末', 'イングランド銀行', 'スイス国立銀行', '中央銀行', '短期金融市場', '英中央銀行', '追加', 'libor', '制度導入', '急増', '緊張', '証券会社', 'ア

808
----------------------------------------
kojin
['イベント', '好調', '注目', '消費動向', '百貨店業界', 'マインド', '個人消費', '動向', '影響', '必要', '悪化', '慎重', '見方', '不安', '圧迫材料', 'mcd', 'tif', 'ティファニー', 'マクドナルド', '下落', '懸念', '推移', '改善傾向', '緩やか', '背景', '雇用所得環境', 'エネルギー高', '低迷', '住宅市場', '圧迫', 'ドル安', '債券高', '市場', '株安', '減速感', '米経済', '関心', 'ceo', '一般世帯', '低下', '内閣府', '数値', '消費者態度指数', '発表', '季節調整値', '水準', '一致', '上昇', '伸び', '内容', '年内', '見通し', '通年', '信頼感指数', '改善', '転換', '動き', '下方修正', '判断', '回復', '地域', '持ち直し', '東北', '共有', '拡大', '減速', '設備投資', '認識', '足もと', 'gdp', '主要', '低調', '押し上げ要因', '価格', '家計', '消費者心理', '示唆', 'リセッション', '可能性', '悪影響', '措置', '説明', '過剰出店', '住宅投資', '改正建築基準法', '理由', '賃金', '指標', '最初', '自動車販売', '大幅', '左右', '情勢', '春闘', '賃上げ達成', '予想', '最高', '物価見通し', '記録', '委員', 'フレディマック', '住宅価格指数', '前期', '米連邦住宅貸付抵当公社', '基調判断', '消費者マインド', '期待', '並み', '円高', '嫌気', '急速', '小売売上高', '浮き彫り', '減退', '米商務省', '要因', 'gdp上', '安定', '底流', '性質', '重要', '横ばい', '維持', '主因', '物価上昇', '言及', '賃金引上げ', '先行き', '増加基調', '投資', '鈍化', '鉱工業生産', 'rics', '最低水準',

----------------------------------------
kabu


['休場', '年間', '株価', '株式市場', '上げ', '下げ幅', '堅調', '石油株', '米株式市場', '縮小', '銀行株', 'vx', 'ダウ', '初日', '大幅下落', '米国株式市場', '低下', '優勢', '円債市場', '材料', '株安', 'オーバーシュート', '反発', '見通し', '週明け', 'スタート', '大幅続落', '大発会', '日経平均株価', '東京株式市場', 'ポジション', '上昇', '動き', '日本株', '銘柄', '見方', '予想レンジ', 'そう', '外部要因', '左右', '為替', '相場', '米株動向', 'ジャスダック市場', 'セブン銀行', '上場', '子会社', '計画', 'ジャスダック', '上場申請', '関係筋', '時価総額', '見込み', 'スローダウン', 'リスク', '円高', '影響', '米国景気', 'アラブ諸国最大', 'サウジアラビア市場', '円高連鎖', '米銀決算', '警戒感', '場面', '打診買い', '日経平均', '金法個人', '反発力', '状況', '配当利回り', '会長', '指摘', '割安感', 'ディフェンシブ銘柄', '医薬品', '消費財', '発表', '米雇用統計', 'ニューヨーク市場', '側面', '渡辺喜美金融担当相', '記者会見', '連動', '閣議後', '債券小', '展開', '東京市場', 'きっかけ', '底入れ', '海外勢', '世界', '大和証券投資信託委託シニアストラテジスト', '長野吉納氏', 'カントリーワイド株', '再開', '取引', '声明発表', '売買停止', '取引再開後', '下落率', '最悪', '不調', '出だし', '下落', '代替投資', '商品市場', '注目', 'コメント', '識者', '通り', '先行', '悪材料', '決算', '金融機関', 'カントリーワイドフィナンシャルcfc', '急落', '悪化', '米大手銀行', '資金繰り', '外国人投資家', '手仕舞い', '終了', '不透明感', '反映', '米市場', '債券高', 'リスクシナリオ', '予想', '参加者', '反転'

youjin
['住宅価格', '比率', 'hd', 'jpモルガン', '予想', '利下げ', 'fomc', 'frb', '調査', '低迷', '住宅市場', '政策', '経済', '検討', '選択肢', '景気後退', '損失', '必要', '予断', '経済見通し', 'ドル', '下落', 'イラン', '世界経済', '米国', '法案', '実現', '退任', '議長', 'ルービン氏', '見通し', '金融市場', '可能性', '日本', 'nrf', '伸び', '指摘', '確率', '増派', '派遣', '欧州', '妥当', '支出', '総裁', '北朝鮮', '核計画', '申告', 'ユーロ', '影響', 'ceo', '問題', '回答', '下方修正', '景気', '減速局面', '米経済', '金利', '減速', '正常', '物価安定', '見方', 'ecb', '議会', '景気対策', '融資', 'オバマ氏', '勝利', 'インディアナ州', 'クリントン氏', '総合経済対策', 'カトリーナ', 'ハリケーン', '緊急対策', '中国', '大統領', '格付け', '決定', 'fdic', '強調', '差し押さえ手続き', '回復', '議論', '逆風', '措置', 'アジア', 'aig', '金融機関', '消費税', '経営者', '深刻', '金融危機', '海外経済', '確信', '状況', '解散', 'ブッシュ政権', '法律', '途上国', '金融ひっ迫', 'ブッシュ大統領', '中川財務金融相', '協調', '対策', '軟調', '国際社会', 'イタルタス通信', 'gm', '政府', 'ロシア', 'エコノミスト', 'プラス成長', '声明', '金融政策', '課題', '取り組み', '米国債', '疑問', '選挙', '情報']
119
----------------------------------------
seiji
['オバマ上院議員', '予備選', '候補者', '勝利', '民主党', '米nbc', '選出', '北朝鮮', '爆弾', '米国', 'オバマ氏', 'ワイオミング州', '事態', '支持率',

In [36]:
data = countVect(data, sepfile_names)

In [37]:
# dict_word = OrderdDict()
#     cat_data["keiki"] = []
#     cat_data["bukka"] = []
#     cat_data["kinri"] = []
#     cat_data["money"] = []
#     cat_data["boueki"] = []
#     cat_data["koyou"] = []
#     cat_data["kojin"] = []
#     cat_data["kainyu"] = []
#     cat_data["oil"] = []
#     cat_data["kabu"] = []
#     cat_data["saiken"] = []
#     cat_data["youjin"] = []
#     cat_data["seiji"] = []
dict_word = {"keiki":[], "bukka":[], "kinri":[], "money":[], "boueki":[], "koyou":[], "kojin":[], "kainyu":[], "oil":[], "kabu":[], "saiken":[], "youjin":[], "seiji":[]}
for d, name in zip(data, sepfile_names):
    dict_word[name] = d

In [25]:
print(dict_word["saiken"])

['石油資源', '購買力', '米商品市場', '原油在庫', '米エネルギー省エネルギー情報局', '週間石油在庫統計', '大豆', '輸入減', '成長ペース', '合成繊維', '吉高執行役員', '素材価格上昇', 'アブドラサウジアラビア国王', 'サウジ国王', '原油不足', '年内成立', '輸入単価', '原油在庫増加', '暖冬', '需要減退懸念', '収支状況', '清水', '資源関連銘柄', '販売額', '石油関連銘柄', 'wti原油先物', '標準油種', '南関東', 'ナイジェリア', 'ベネズエラ', '供給不安定', 'nimex', 'ニューヨーク商業取引所', '関連業種', 'プラチナ', '農産物', 'リセッション懸念', 'ジェフリーズcrb指数', '先物相場', '米国向け輸出', '電話インタビュー', '増産決定', '統制外', '小幅増産', 'opec加盟諸国', 'アジュモゴビア石油担当相', 'ヌアイミ石油鉱物資源相', '十分供給', '企業収益圧迫要因', '国際商品価格', 'コストアップ', 'ヒーティングオイル価格', '付け急騰', 'ドル安インフレ懸念', '流動性供給策', '見直し余地', 'トラック', '排ガス規制', '新車代替需要', '国際経済', 'ヘリル議長', '石油輸出国機構', 'cftc', 'nymexエネルギー先物取り組み', '投機的ポジション', '米商品先物取引委員会', '金属', '鉱山', '作物', 'ニューヨーク商品先物取引所', '米金先物', '循環メカニズム', '工業株', '住宅問題', '週間在庫統計', '輸入減少', 'クウェート侵攻', '混乱期', '個別物色方向', '国際商品', '米フィラデルフィア地区連銀', 'ief', '国際エネルギーフォーラム', '価格変動', '原油価格水準', '生産国消費国', '世界経済成長見通し', '原燃料', '米ドル安', 'コーンミール', '貴金属相場', '非鉄金属', '収益環境', '浜口氏', '外的ショック', '金利据え置き観測', '石油会社', '大手商社', '資源開発', '関連株', '金属価格', '石油関連株', '幾分圧迫', '小

In [40]:
writeJSON(dict_word)

print("-----頻出語の抽出完了-----")
# cv.tfIdf(data)
# countvectorizer(出現頻度推定)
# cv.countVect(df, sepfile_names[:-1])

-----頻出語の抽出完了-----
